In [108]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error 

1. Выбрать и загрузить датасет для задачи регрессии (можно взять из UCI), содержащий как вещественные, так и категориальные признаки. Очень желательно, чтобы количество наблюдений в датасете было больше 1к. В данной работе в качестве метрик качества можно использовать R^2, MAE, RMSE.
Временно убрать категориальные признаки и обучить модель линейной регрессии только на оставшихся вещественных признаках. Знатоки могут попробовать OLS из statsmodels вместо LinearRegression из sklearn;

In [109]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("harlfoxem/housesalesprediction")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Andrjusha\.cache\kagglehub\datasets\harlfoxem\housesalesprediction\versions\1


In [110]:
data = pd.read_csv(os.path.join(path, "kc_house_data.csv"))
data = data.drop(['id', 'date'], axis=1)
data

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [ ]:
target = 'price'
categorical = data.columns[data.nunique() < 13].tolist()
categorical.remove('floors')
print(categorical)
numerical = [col for col in data.columns if col not in categorical + [target]]

X_num = data[numerical]
y = data[target]

X_train_num, X_test_num, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=11)

lin_reg = LinearRegression()
lin_reg.fit(X_train_num, y_train)

y_pred = lin_reg.predict(X_test_num)
r2_lin_num = r2_score(y_test, y_pred)
mae_lin_num = mean_absolute_error(y_test, y_pred)
rmse_lin_num = root_mean_squared_error(y_test, y_pred)

print(f"R^2: {r2_lin_num}, MAE: {mae_lin_num}, RMSE: {rmse_lin_num}")

['waterfront', 'view', 'condition', 'grade']
R^2: 0.628182303771907, MAE: 140657.86841975243, RMSD: 207619.52754942595


2. На этих же вещественных признаках обучить модель градиентного бустинга;

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor  

gb_reg = GradientBoostingRegressor(random_state=11)
gb_reg.fit(X_train_num, y_train)

y_pred_gb = gb_reg.predict(X_test_num)
r2_gb_num = r2_score(y_test, y_pred_gb)
mae_gb_num = mean_absolute_error(y_test, y_pred_gb)
rmse_gb_num = root_mean_squared_error(y_test, y_pred_gb)

print(f"R^2: {r2_gb_num}, MAE: {mae_gb_num}, RMSE: {rmse_gb_num}")

R^2: 0.8106234450768859, MAE: 87263.11376229166, RMSD: 148172.04555806468


3. Для категориальных признаков применить one-hot, label и target encoding. Для каждой из кодировок обучить линейную регрессию и тот же бустинг (не только на закодированных категориальных, а на всех данных вместе, включая изначальные вещественные признаки);

In [113]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, TargetEncoder

In [114]:
X = data.drop(target, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [115]:
def get_metrics(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return {
        'R2': r2_score(y_test, y_pred),
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': root_mean_squared_error(y_test, y_pred)
    }


In [116]:
# One-Hot Encoding
ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore') 
X_train_ohe_cat = pd.DataFrame(ohe.fit_transform(X_train[categorical]), index=X_train.index)
X_test_ohe_cat = pd.DataFrame(ohe.transform(X_test[categorical]), index=X_test.index)
X_train_ohe = pd.concat([X_train[numerical], X_train_ohe_cat], axis=1)
X_test_ohe = pd.concat([X_test[numerical], X_test_ohe_cat], axis=1)
X_train_ohe.columns = X_train_ohe.columns.astype(str)
X_test_ohe.columns  = X_test_ohe.columns.astype(str)

lin_metrics_ohe = get_metrics(LinearRegression(), X_train_ohe, X_test_ohe, y_train, y_test)
gb_metrics_ohe = get_metrics(GradientBoostingRegressor(random_state=42), X_train_ohe, X_test_ohe, y_train, y_test)


In [117]:
# Label Encoding 
le_dict = {col: LabelEncoder().fit(X_train[col]) for col in categorical}
X_train_le = X_train.copy()
X_test_le = X_test.copy()
for col in categorical:
    X_train_le[col] = le_dict[col].transform(X_train[col])
    X_test_le[col] = le_dict[col].transform(X_test[col])

lin_metrics_le = get_metrics(LinearRegression(), X_train_le, X_test_le, y_train, y_test)
gb_metrics_le = get_metrics(GradientBoostingRegressor(random_state=42), X_train_le, X_test_le, y_train, y_test)

In [118]:
# Target Encoding
te = TargetEncoder(target_type='continuous')
X_train_te = X_train.copy()
X_test_te = X_test.copy()
X_train_te[categorical] = te.fit_transform(X_train[categorical], y_train)
X_test_te[categorical] = te.transform(X_test[categorical])

lin_metrics_te = get_metrics(LinearRegression(), X_train_te, X_test_te, y_train, y_test)
gb_metrics_te = get_metrics(GradientBoostingRegressor(random_state=11), X_train_te, X_test_te, y_train, y_test)

4. Не применяя кодировки, обучить catboost на исходных данных (надо разобраться как ему подать вещественные и категориальные признаки, чтобы всё заработало);

In [119]:
from catboost import CatBoostRegressor, Pool

train_pool = Pool(X_train, y_train, cat_features=categorical)
test_pool = Pool(X_test, y_test, cat_features=categorical)

cat_reg = CatBoostRegressor(random_state=11, verbose=False)  
cat_reg.fit(train_pool)

y_pred_cat = cat_reg.predict(test_pool)
r2_cat = r2_score(y_test, y_pred_cat)
mae_cat = mean_absolute_error(y_test, y_pred_cat)
rmse_cat = root_mean_squared_error(y_test, y_pred_cat)

print(f"R^2: {r2_cat}, MAE: {mae_cat}, RMSE: {rmse_cat}")

R^2: 0.8998421704118554, MAE: 66616.76501411937, RMSE: 123050.9300408502


In [120]:
cat_reg_2 = CatBoostRegressor(random_state=11, verbose=False)  
cat_reg_2.fit(X_train, y_train)

y_pred_cat_2 = cat_reg_2.predict(X_test)
r2_cat_2 = r2_score(y_test, y_pred_cat_2)
mae_cat_2 = mean_absolute_error(y_test, y_pred_cat_2)
rmse_cat_2 = root_mean_squared_error(y_test, y_pred_cat_2)

print(f"R^2: {r2_cat_2}, MAE: {mae_cat_2}, RMSE: {rmse_cat_2}")

R^2: 0.9026368801843924, MAE: 65703.63417345696, RMSE: 121322.03570989883


5. Собрать все результаты из предыдущих пунктов (метрики) в один датафрейм и сделать выводы.

In [121]:
# Сбор метрик
metrics_dict = {
    'Linear_NumOnly': {'R2': r2_lin_num, 'MAE': mae_lin_num, 'RMSE': rmse_lin_num},
    'GB_NumOnly': {'R2': r2_gb_num, 'MAE': mae_gb_num, 'RMSE': rmse_gb_num},
    'Linear_OneHot': lin_metrics_ohe,
    'GB_OneHot': gb_metrics_ohe,
    'Linear_Label': lin_metrics_le,
    'GB_Label': gb_metrics_le,
    'Linear_Target': lin_metrics_te,
    'GB_Target': gb_metrics_te,
    'CatBoost_Raw': {'R2': r2_cat, 'MAE': mae_cat, 'RMSE': rmse_cat}
}

metrics_df = pd.DataFrame(metrics_dict).T
print(metrics_df)

                      R2            MAE           RMSE
Linear_NumOnly  0.628182  140657.868420  207619.527549
GB_NumOnly      0.810623   87263.113762  148172.045558
Linear_OneHot   0.735611  121727.612987  199923.778280
GB_OneHot       0.848963   84691.929880  151107.084773
Linear_Label    0.701197  127498.209981  212537.184644
GB_Label        0.851987   81326.857144  149586.605715
Linear_Target   0.717029  124654.749560  206830.087577
GB_Target       0.855065   82190.939976  148022.849115
CatBoost_Raw    0.899842   66616.765014  123050.930041


Выводы: 
Линейная регрессия показывает худший уровень качества.
Добавление категориальных признаков с кодировками улучшает качество как линейной регрессии, так и бустинга.
Catboost показывает лучшее качество среди моделей.